**This notebook contains a script that creates saves and tracks POE documents**

**Add local Library to Path**

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

**Import Library**

In [2]:
import pandas as pd
import re
from local_library import create_folder
from local_library import import_worksheet
from local_library import export_worksheet
from local_library import get_documents_metadata

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-f1d5s7cpjrj54u2rqk1ufh9pevguqoap.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


**Import Data**

In [17]:
#import cohort data
cohort_df = import_worksheet("Students 02-08-22", "Students (3).csv")

**Check 2021 columns**

In [28]:
#cohort_df['Email Address']


**Update Column Names**

In [20]:
column_names = {"First Name":"learner_name", "Last Name": "learner_surname",
                "Email Address":"learner_email", "Middle Name(s)":"middle_name"
               }

cohort_df = cohort_df.rename(columns=column_names)

**Create Fullname**

In [21]:
cohort_df['middle_name'] = cohort_df['middle_name'].fillna("")
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['middle_name'].str.strip().str.lower()
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.title()
cohort_df['learner_fullname'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['learner_surname'].str.strip().str.lower()
cohort_df['learner_fullname'] = [re.sub(' +', ' ', fullname) for fullname in cohort_df['learner_fullname']]
cohort_df['learner_fullname'] = cohort_df['learner_fullname'].str.title()


**Subset Useful Columns**

In [22]:
column_names = ['Username', 'Campus', 'learner_email', 'learner_fullname']

cohort_df = cohort_df[cohort_df['Registration Status'] == 'Active']
cohort_df = cohort_df[cohort_df['Seta Eligible'] != 'No']
cohort_df = cohort_df[column_names]

**Create Folder ID Column**

In [5]:
#cohort_df['folder_id'] = ''
cohort_df.columns

Index(['Username', 'Campus', 'learner_email', 'learner_fullname', 'folder_id',
       'poe_instructions', 'learner_contract', 'appeals_procedure',
       'assessment_contract', 'preassessment_meeting'],
      dtype='object')

**Create Folders**

In [24]:
#for index, row in cohort_df.iterrows():
#    parent_id = create_folder("1NANfcxZ4fZ_yqMTPLgZO2Cm5ZPb_Gy07", row['learner_fullname'])
#    cohort_df.loc[cohort_df['learner_fullname'] == row['learner_fullname'], 'folder_id'] = parent_id

**Add template IDS**

In [25]:
cohort_df['poe_instructions'] = 'feb67241-52f8-456c-917e-bdd0526619ee'
cohort_df['learner_contract'] = '900be1ba-0b96-4386-a904-fb67069f168e'
cohort_df['appeals_procedure'] = '589d226b-1c7e-441f-b99e-1d30a2d37761'
cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'assessment_contract'] = '215045cd-c510-4d04-8280-ab8dbf0f7d64'
cohort_df.loc[cohort_df['Campus'] != 'Johannesburg', 'assessment_contract'] = 'aeede3cc-6496-49be-82b0-a959911c522b'
cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'preassessment_meeting'] = '708875f6-b6a8-4876-aa3a-e9104b481ae5'
cohort_df.loc[cohort_df['Campus'] != 'Johannesburg', 'preassessment_meeting'] = '08e3cc74-112f-4cd4-ad37-05bc726a5b96'

In [16]:
#Declaration of Authenticity
#cohort_df['authenticity_declaration_status'] = ''
cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'authenticity_declaration'] = 'd2308911-b247-4957-b865-e76f504ef91f'
cohort_df.loc[cohort_df['Campus'] != 'Johannesburg', 'authenticity_declaration'] = '78c858be-3acf-4c37-987d-672536c3196e'

In [17]:
#Program Evaluation
#cohort_df['program_evaluation_status'] = ''
cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'program_evaluation'] = 'ecae0443-8f3c-44dd-b2ee-7194a4fde67b'
cohort_df.loc[cohort_df['Campus'] != 'Johannesburg', 'program_evaluation'] = 'f97eb0a8-97a9-40c7-84df-29aaee7e9893'

In [18]:
#Logbook
#cohort_df['logbook_status'] = ''
cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'logbook'] = 'f8c63d3d-fd0f-413f-a028-35dd9990b30d'
cohort_df.loc[cohort_df['Campus'] != 'Johannesburg', 'logbook'] = '911ce292-ec38-4ef9-a8a0-b48b1adbef5f'

**Set Column Status**

In [9]:
#document_names = ['preassessment_meeting',  'poe_instructions', 'learner_contract','appeals_procedure', 'assessment_contract']

#for document in document_names:
#    cohort_df[f"{document}_status"] = ''

**Save Database to Spreadsheet**

In [26]:
#export_worksheet("POE Compilation 2021","POE Documents", cohort_df)

**Import POE Template Data**

In [3]:
cohort_df = import_worksheet("POE Compilation 2021","POE Documents")

**Run Main Function**

In [8]:
#create document names 'authenticity_declaration','program_evaluation', 'logbook',  
document_names = ['preassessment_meeting',  'poe_instructions', 'learner_contract','appeals_procedure','assessment_contract']

for document_name in document_names:
    get_documents_metadata(cohort_df, document_name, "learner_email")
    print(document_name)

assessment_contract


**Save to POE tracking sheet**

In [10]:
#export_worksheet("2021 SETA Submission Tracking Sheet","POE Documents Data Dump", cohort_df[['Username', 'Campus',
#                'poe_instructions_status', 'learner_contract_status', 'appeals_procedure_status', 'assessment_contract_status',
#                'preassessment_meeting_status', 'authenticity_declaration_status', 'program_evaluation_status', 'logbook_status']])


**Save to POE Follow sheet**

In [11]:
#export_worksheet("POE Compilation 2021","POE Documents", cohort_df)
